In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True

#Import modules: 

#from cython.parallel import prange, parallel
from libc.stdlib cimport rand, RAND_MAX, calloc, malloc, realloc, free, abort

######################### Define the Data Structure ############################

cdef struct Parameters:
    #Pointer for Y data array
    double* Y
    #size of the array
    int* Size

################ Support Functions for Monte-Carlo Function ##################

#Create a function that allocates the memory and verifies integrity
cdef int alloc_struct(Parameters* data, int* N, unsigned int flag, int Mem_Int) nogil:
    
    #fill in the size of the array
    data.Size = N
    
    #allocate the data array initially
    if flag==0:
        data.Y = <double*> calloc(N[0], sizeof(double))
    #reallocate the data array
    else:
        data.Y = <double*> realloc(data.Y, N[0] * sizeof(double))
    
    #If the elements of the struct are not properly allocated, destory it and update the Integrity variable
    if N[0]!=0 and data.Y==NULL:
        
        #return the memory to system
        destroy_struct(data)
        
        #update the memory integrity variable to False
        Mem_Int = False
        
        return Mem_Int
    
    else: return Mem_Int

#Create the destructor of the struct to return memory to system
cdef void destroy_struct(Parameters* data) nogil:
    free(data.Y)
    free(data)
    
#This function fills in the Y observed variable with discreet 0/1
cdef void Y_fill(Parameters* data, double p_true) nogil:
    
    cdef:
        Py_ssize_t i
        double y
    
    for i in range(data.Size[0]):
        
        y = rand()/<double>RAND_MAX
        
        if y <= p_true:
            data.Y[i] = 1
    
cpdef void TEST(int[::1] Samples, double p_true, Py_ssize_t Sims) nogil:
    
    #Define variables and pointers
    cdef:
        #Data Structure
        Parameters* Data
            
        #iterators
        Py_ssize_t i, j
        
        #Variables
        int Mem_Int=True, N = Samples.shape[0]
        
    #allocate the struct dynamically
    Data = <Parameters*> malloc(sizeof(Parameters))
    
    #verify memory integrity of the struct
    if Data==NULL: abort()
    
    try:
        
        for i in range(N):
            
            #with gil:
                #print('New Structure')

            #allocate the elements of the struct (if i>0, reallocate)
            Mem_Int = alloc_struct(Data, &Samples[i], i, Mem_Int)

            #verify memory integrity (exit if memory not properly allocated)
            if Mem_Int==False: abort() 
            
            for j in range(Sims):
                
                #fill in the struct
                Y_fill(Data, p_true)
                
    finally:
        destroy_struct(Data)

In [4]:
#First we will recreate the first cell in the matlab example in python

#Sample Sizes
N = np.array([5,50,500,5000], dtype='i')

#Parameters
T = 1000
p_true = 0.05

#Array of the outputs from the MC
# p_hat = np.empty((N.size,T), dtype='d')
# p_hat.fill(np.nan)

In [5]:
%timeit TEST(N,p_true,T)

10 loops, best of 3: 39.6 ms per loop
